<a href="https://colab.research.google.com/github/sumanyurosha/tensorflow-specialization/blob/master/Hands-on%20ML/chapter10/Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

In [2]:
housing_data = fetch_california_housing()
x, y = housing_data.data, housing_data.target

print(x.shape, y.shape)

(20640, 8) (20640,)


In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train)

print(x_train.shape, x_valid.shape, x_test.shape)

(11610, 8) (3870, 8) (5160, 8)


In [4]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)

In [5]:
def build_model(n_hidden=1, n_neurons=30, activation='relu', learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(units=n_neurons, activation=activation))
    model.add(keras.layers.Dense(1))
    model.compile(optimizer=keras.optimizers.SGD(lr=learning_rate), loss='mse')
    return model

In [6]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

params = {
    'n_hidden' : [1, 2, 3, 4],
    'n_neurons' : np.arange(1, 100),
    'learning_rate': reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, params, n_iter=10, cv=3)
rnd_search_cv.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid),
                  callbacks=keras.callbacks.EarlyStopping(patience=10))

Epoch 1/100
242/242 [==============================] - 1s 3ms/step - loss: 1.2557 - val_loss: 0.8446
Epoch 2/100
242/242 [==============================] - 1s 2ms/step - loss: 0.7690 - val_loss: 0.7327
Epoch 3/100
242/242 [==============================] - 1s 2ms/step - loss: 0.6982 - val_loss: 0.6838
Epoch 4/100
242/242 [==============================] - 1s 2ms/step - loss: 0.6570 - val_loss: 0.6527
Epoch 5/100
242/242 [==============================] - 1s 2ms/step - loss: 0.6266 - val_loss: 0.6242
Epoch 6/100
242/242 [==============================] - 1s 2ms/step - loss: 0.6029 - val_loss: 0.5984
Epoch 7/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5798 - val_loss: 0.5739
Epoch 8/100
242/242 [==============================] - 1s 3ms/step - loss: 0.5572 - val_loss: 0.5476
Epoch 9/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5364 - val_loss: 0.5242
Epoch 10/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5156 - val_lo